In [1]:
getwd()
setwd("/liulab/galib/dlbcl_manuscript/")
library(tidyverse)
library(Seurat)
library(ktplots)
library(ComplexHeatmap)
library(circlize)
library(Polychrome)
library(RColorBrewer)
library(pals)
library(SeuratDisk)
source('./scripts/scplot.R')

[1] "/liulab/galib/dlbcl_manuscript/scripts"

Warning message:
“package ‘tidyverse’ was built under R version 4.1.3”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.0      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.1 

Warning message:
“package ‘tidyr’ was built under R version 4.1.2”
Warning message:
“package ‘readr’ was built under R version 4.1.2”
Warning message:
“package ‘forcats’ was built under R version 4.1.3”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching SeuratObject

Attaching sp

Warning message:
“package ‘ComplexHeatmap’ was built under R version 4.1.3”
Loading required package: grid

ComplexHeatmap version 2.10.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.g

## reformat data to input of cellphoneDB

In [2]:
merged<- readRDS("./data/objects/merged/all_cells.obj")

In [ ]:
# convert mouse gene name to human gene name
count_tbl<- merged@assays$RNA@counts
all_genes<- rownames(count_tbl)
mm_hs_genes<- readRDS("./data/cellphoneDB/GeneID_Annotation_mmu_hsa.rds")
mm_hs_genes$mmu_symbol<- mm_hs_genes$mmu_symbol  %>%  str_to_title()
mm_hs_genes<- mm_hs_genes[!mm_hs_genes$mmu_symbol  %>% duplicated(),]  %>% filter(hsa_symbol != "")
shared_genes<- intersect(mm_hs_genes$mmu_symbol, all_genes)
shared_genes  %>% length()
rownames(mm_hs_genes)<- mm_hs_genes$mmu_symbol
table(rownames(count_tbl)== "")
new_count_tbl<- count_tbl[shared_genes,]
rownames(new_count_tbl)<- mm_hs_genes[shared_genes, "hsa_symbol"]
rownames(new_count_tbl)  %>% duplicated()  %>% table()
expr_sum<- rowSums(new_count_tbl)
df<- data.frame(exprSum = expr_sum, idx = c(1:length(expr_sum)), genes = names(expr_sum))  %>% arrange(-exprSum)
deduplicated_count<- new_count_tbl[df[!df$genes  %>% duplicated(),"idx"],]
deduplicated_count  %>% dim()
rownames(deduplicated_count)<- rownames(deduplicated_count)  %>% toupper()
cellphoneDB_obj <- CreateSeuratObject(counts = deduplicated_count, project = "cellphoneDB")

In [ ]:
cd3_pos_cd4_neg<- readRDS("./data/objects/cd3_pos_cd4_neg/cd3_pos_cd4_neg_final.rds")
cd3_pos_cd8_neg<- readRDS("./data/objects/cd3_pos_cd8_neg/cd3_pos_cd8_neg_final.rds")
non_B_non_T<- readRDS("./data/objects/non_B_non_T/non_B_non_T_c18_subclustered.rds")
B_cell<- readRDS("./data/objects/B_cell/B_cell_annotated_add_color.rds")

In [ ]:
cellphoneDB_obj$annotation = NA
cellphoneDB_obj@meta.data[rownames(cd3_pos_cd4_neg@meta.data), "annotation"] = as.character(cd3_pos_cd4_neg$new_annotation)
cellphoneDB_obj@meta.data[rownames(cd3_pos_cd8_neg@meta.data), "annotation"] = as.character(cd3_pos_cd8_neg$new_annotation)
cellphoneDB_obj@meta.data[rownames(non_B_non_T@meta.data), "annotation"] = as.character(non_B_non_T$seurat_clusters)
cellphoneDB_obj@meta.data[rownames(B_cell@meta.data), "annotation"] = as.character(B_cell$new_annotation)

cellphoneDB_obj@meta.data[rownames(cd3_pos_cd4_neg@meta.data), "space"] = "cd3_pos_cd4_neg"
cellphoneDB_obj@meta.data[rownames(cd3_pos_cd8_neg@meta.data), "space"] = "cd3_pos_cd8_neg"
cellphoneDB_obj@meta.data[rownames(non_B_non_T@meta.data), "space"] = "non_B_non_T"
cellphoneDB_obj@meta.data[rownames(B_cell@meta.data), "space"] = "B_cell"

cellphoneDB_obj@meta.data[rownames(cd3_pos_cd4_neg@meta.data), "genotype"] = cd3_pos_cd4_neg$genotype
cellphoneDB_obj@meta.data[rownames(cd3_pos_cd8_neg@meta.data), "genotype"] = cd3_pos_cd8_neg$genotype
cellphoneDB_obj@meta.data[rownames(non_B_non_T@meta.data), "genotype"] = non_B_non_T$genotype
cellphoneDB_obj@meta.data[rownames(B_cell@meta.data), "genotype"] = B_cell$genotype

cellphoneDB_obj@meta.data[rownames(cd3_pos_cd4_neg@meta.data), "age"] = cd3_pos_cd4_neg$age
cellphoneDB_obj@meta.data[rownames(cd3_pos_cd8_neg@meta.data), "age"] = cd3_pos_cd8_neg$age
cellphoneDB_obj@meta.data[rownames(non_B_non_T@meta.data), "age"] = non_B_non_T$age
cellphoneDB_obj@meta.data[rownames(B_cell@meta.data), "age"] = B_cell$age

saveRDS(cellphoneDB_obj, file = "./data/cellphoneDB/cellphoneDB_orig.obj")

In [ ]:
cellphoneDB_bcl6_6mos<- cellphoneDB_obj@meta.data  %>% filter(genotype == "Bcl6tg/+")  %>% filter(age == "6mos")  %>% rownames()
cellphoneDB_bcl6_14mos<- cellphoneDB_obj@meta.data  %>% filter(genotype == "Bcl6tg/+")  %>% filter(age == "14mos")  %>% rownames()
cellphoneDB_bcl6_18mos<- cellphoneDB_obj@meta.data  %>% filter(genotype == "Bcl6tg/+")  %>% filter(age == "18mos")  %>% rownames()
cellphoneDB_bcl6_sick<- cellphoneDB_obj@meta.data  %>% filter(genotype == "Bcl6tg/+")  %>% filter(age == "sick")  %>% rownames()

In [ ]:
cellphoneDB_bcl6_6mos_obj<- subset(cellphoneDB_obj, cells = cellphoneDB_bcl6_6mos)
cellphoneDB_bcl6_14mos_obj<- subset(cellphoneDB_obj, cells = cellphoneDB_bcl6_14mos)
cellphoneDB_bcl6_18mos_obj<- subset(cellphoneDB_obj, cells = cellphoneDB_bcl6_18mos)
cellphoneDB_bcl6_sick_obj<- subset(cellphoneDB_obj, cells = cellphoneDB_bcl6_sick)

In [ ]:
write10xCounts(x = cellphoneDB_bcl6_6mos_obj@assays$RNA@counts, path = "./data/cellphoneDB/10X_format/bcl6_6mos/")
write10xCounts(x = cellphoneDB_bcl6_14mos_obj@assays$RNA@counts, path = "./data/cellphoneDB/10X_format/bcl6_14mos/")
write10xCounts(x = cellphoneDB_bcl6_18mos_obj@assays$RNA@counts, path = "./data/cellphoneDB/10X_format/bcl6_18mos/")
write10xCounts(x = cellphoneDB_bcl6_sick_obj@assays$RNA@counts, path = "./data/cellphoneDB/10X_format/bcl6_sick")

In [ ]:
cellphoneDB_bcl6_6mos_meta<- data.frame(Cell = rownames(cellphoneDB_bcl6_6mos_obj@meta.data), 
                                        cell_type = cellphoneDB_bcl6_6mos_obj@meta.data$new_annotation)
cellphoneDB_bcl6_14mos_meta<- data.frame(Cell = rownames(cellphoneDB_bcl6_14mos_obj@meta.data), 
                                        cell_type = cellphoneDB_bcl6_14mos_obj@meta.data$new_annotation)

cellphoneDB_bcl6_18mos_meta<- data.frame(Cell = rownames(cellphoneDB_bcl6_18mos_obj@meta.data), 
                                        cell_type = cellphoneDB_bcl6_18mos_obj@meta.data$new_annotation)

cellphoneDB_bcl6_sick_meta<- data.frame(Cell = rownames(cellphoneDB_bcl6_sick_obj@meta.data), 
                                        cell_type = cellphoneDB_bcl6_sick_obj@meta.data$new_annotation)

write.table(cellphoneDB_bcl6_6mos_meta, file = "./data/cellphoneDB/10X_format/bcl6_6mos_meta.txt", sep = "\t", 
            col.names = TRUE, row.names = FALSE, quote = FALSE)
write.table(cellphoneDB_bcl6_14mos_meta, file = "./data/cellphoneDB/10X_format/bcl6_14mos_meta.txt", sep = "\t", 
            col.names = TRUE, row.names = FALSE, quote = FALSE)
write.table(cellphoneDB_bcl6_18mos_meta, file = "./data/cellphoneDB/10X_format/bcl6_18mos_meta.txt", sep = "\t", 
            col.names = TRUE, row.names = FALSE, quote = FALSE)
write.table(cellphoneDB_bcl6_sick_meta, file = "./data/cellphoneDB/10X_format/bcl6_sick_meta.txt", sep = "\t", 
            col.names = TRUE, row.names = FALSE, quote = FALSE)

In [2]:
# read bcl6 single
ph_bcl6_6mos_pvals <- read.delim("./data/cellphoneDB/output/bcl6_6mos_out/pvalues.txt", check.names = FALSE)
ph_bcl6_6mos_means <- read.delim("./data/cellphoneDB/output/bcl6_6mos_out/means.txt", check.names = FALSE)
ph_bcl6_14mos_pvals <- read.delim("./data/cellphoneDB/output/bcl6_14mos_out/pvalues.txt", check.names = FALSE)
ph_bcl6_14mos_means <- read.delim("./data/cellphoneDB/output/bcl6_14mos_out/means.txt", check.names = FALSE)
ph_bcl6_18mos_pvals <- read.delim("./data/cellphoneDB/output/bcl6_18mos_out/pvalues.txt", check.names = FALSE)
ph_bcl6_18mos_means <- read.delim("./data/cellphoneDB/output/bcl6_18mos_out/means.txt", check.names = FALSE)
ph_bcl6_sick_pvals <- read.delim("./data/cellphoneDB/output/bcl6_sick_out/pvalues.txt", check.names = FALSE)
ph_bcl6_sick_means <- read.delim("./data/cellphoneDB/output/bcl6_sick_out/means.txt", check.names = FALSE)

# read bcl6 cd70 double
ph_bcl6_double_6mos_pvals <- read.delim("./data/cellphoneDB/output/bcl6_double_6mos_out/pvalues.txt", check.names = FALSE)
ph_bcl6_double_6mos_means <- read.delim("./data/cellphoneDB/output/bcl6_double_6mos_out/means.txt", check.names = FALSE)
ph_bcl6_double_14mos_pvals <- read.delim("./data/cellphoneDB/output/bcl6_double_14mos_out/pvalues.txt", check.names = FALSE)
ph_bcl6_double_14mos_means <- read.delim("./data/cellphoneDB/output/bcl6_double_14mos_out/means.txt", check.names = FALSE)
ph_bcl6_double_18mos_pvals <- read.delim("./data/cellphoneDB/output/bcl6_double_18mos_out/pvalues.txt", check.names = FALSE)
ph_bcl6_double_18mos_means <- read.delim("./data/cellphoneDB/output/bcl6_double_18mos_out/means.txt", check.names = FALSE)
ph_bcl6_double_sick_pvals <- read.delim("./data/cellphoneDB/output/bcl6_double_sick_out/pvalues.txt", check.names = FALSE)
ph_bcl6_double_sick_means <- read.delim("./data/cellphoneDB/output/bcl6_double_sick_out/means.txt", check.names = FALSE)

# read wt
ph_wt_6mos_pvals <- read.delim("./data/cellphoneDB/output/wt_6mos_out/pvalues.txt", check.names = FALSE)
ph_wt_6mos_means <- read.delim("./data/cellphoneDB/output/wt_6mos_out/means.txt", check.names = FALSE)
ph_wt_14mos_pvals <- read.delim("./data/cellphoneDB/output/wt_14mos_out/pvalues.txt", check.names = FALSE)
ph_wt_14mos_means <- read.delim("./data/cellphoneDB/output/wt_14mos_out/means.txt", check.names = FALSE)
ph_wt_18mos_pvals <- read.delim("./data/cellphoneDB/output/wt_18mos_out/pvalues.txt", check.names = FALSE)
ph_wt_18mos_means <- read.delim("./data/cellphoneDB/output/wt_18mos_out/means.txt", check.names = FALSE)

### 1. cd3_pos_cd4_neg, cd3_pos_cd8_neg, and non_B_non_T respectively with B_cell

In [3]:
cd3_pos_cd4_neg_clusters<- c("cd3_pos_cd4_neg_Cycling", "cd3_pos_cd4_neg_CTLs")
B_cell_clusters<- c("B_cell_Atypical", "B_cell_Age-associated/autoimmune", "B_cell_Cycling", "B_cell_Plasmablasts")

cd3_pos_cd4_neg_B_cell_interacts<- c()
B_cell_cd3_pos_cd4_neg_interacts<- c()

for (c1 in cd3_pos_cd4_neg_clusters){
    for (c2 in B_cell_clusters){
        
        interact<- paste0(c1, "|", c2)
        inverse_interact<- paste0(c2, "|", c1)
        
        cd3_pos_cd4_neg_B_cell_interacts<- c(cd3_pos_cd4_neg_B_cell_interacts, interact)
        B_cell_cd3_pos_cd4_neg_interacts<- c(B_cell_cd3_pos_cd4_neg_interacts, inverse_interact)
    }
}

cd3_pos_cd4_neg_B_cell_interacts
sum(colnames(ph_wt_6mos_pvals) %in% cd3_pos_cd4_neg_B_cell_interacts) == length(cd3_pos_cd4_neg_B_cell_interacts)

B_cell_cd3_pos_cd4_neg_interacts
sum(colnames(ph_wt_6mos_pvals) %in% B_cell_cd3_pos_cd4_neg_interacts) == length(B_cell_cd3_pos_cd4_neg_interacts)

cd3_pos_cd8_neg_clusters<- c("cd3_pos_cd8_neg_Tregs", "cd3_pos_cd8_neg_Follicular Tregs", 
                             "cd3_pos_cd8_neg_Cycling", "cd3_pos_cd8_neg_CTLs")
B_cell_clusters<- c("B_cell_Atypical", "B_cell_Age-associated/autoimmune", 
                    "B_cell_Cycling", "B_cell_Plasmablasts")

cd3_pos_cd8_neg_B_cell_interacts<- c()
B_cell_cd3_pos_cd8_neg_interacts<- c()

for (c1 in cd3_pos_cd8_neg_clusters){
    for (c2 in B_cell_clusters){
        
        interact<- paste0(c1, "|", c2)
        inverse_interact<- paste0(c2, "|", c1)
        
        cd3_pos_cd8_neg_B_cell_interacts<- c(cd3_pos_cd8_neg_B_cell_interacts, interact)
        B_cell_cd3_pos_cd8_neg_interacts<- c(B_cell_cd3_pos_cd8_neg_interacts, inverse_interact)
    }
}

cd3_pos_cd8_neg_B_cell_interacts
# cd3_pos_cd8_neg_B_cell_interacts[!cd3_pos_cd8_neg_B_cell_interacts %in% colnames(ph_wt_6mos_pvals)]
sum(colnames(ph_wt_6mos_pvals) %in% cd3_pos_cd8_neg_B_cell_interacts) == length(cd3_pos_cd8_neg_B_cell_interacts)

B_cell_cd3_pos_cd8_neg_interacts
sum(colnames(ph_wt_6mos_pvals) %in% B_cell_cd3_pos_cd8_neg_interacts) == length(B_cell_cd3_pos_cd8_neg_interacts)

non_B_non_T_clusters<- c("non_B_non_T_Macrophages", "non_B_non_T_NK cells", 
                             "non_B_non_T_Cycling", "non_B_non_T_Neutrophils")
B_cell_clusters<- c("B_cell_Atypical", "B_cell_Age-associated/autoimmune", 
                    "B_cell_Cycling", "B_cell_Plasmablasts")

non_B_non_T_B_cell_interacts<- c()
B_cell_non_B_non_T_interacts<- c()

for (c1 in non_B_non_T_clusters){
    for (c2 in B_cell_clusters){
        
        interact<- paste0(c1, "|", c2)
        inverse_interact<- paste0(c2, "|", c1)
        
        non_B_non_T_B_cell_interacts<- c(non_B_non_T_B_cell_interacts, interact)
        B_cell_non_B_non_T_interacts<- c(B_cell_non_B_non_T_interacts, inverse_interact)
    }
}

non_B_non_T_B_cell_interacts
# non_B_non_T_B_cell_interacts[!non_B_non_T_B_cell_interacts %in% colnames(ph_wt_6mos_pvals)]
sum(colnames(ph_wt_6mos_pvals) %in% non_B_non_T_B_cell_interacts) == length(non_B_non_T_B_cell_interacts)

B_cell_non_B_non_T_interacts
sum(colnames(ph_wt_6mos_pvals) %in% B_cell_non_B_non_T_interacts) == length(B_cell_non_B_non_T_interacts)

[1] "cd3_pos_cd4_neg_Cycling|B_cell_Atypical"                 
[2] "cd3_pos_cd4_neg_Cycling|B_cell_Age-associated/autoimmune"
[3] "cd3_pos_cd4_neg_Cycling|B_cell_Cycling"                  
[4] "cd3_pos_cd4_neg_Cycling|B_cell_Plasmablasts"             
[5] "cd3_pos_cd4_neg_CTLs|B_cell_Atypical"                    
[6] "cd3_pos_cd4_neg_CTLs|B_cell_Age-associated/autoimmune"   
[7] "cd3_pos_cd4_neg_CTLs|B_cell_Cycling"                     
[8] "cd3_pos_cd4_neg_CTLs|B_cell_Plasmablasts"

[1] TRUE

[1] "B_cell_Atypical|cd3_pos_cd4_neg_Cycling"                 
[2] "B_cell_Age-associated/autoimmune|cd3_pos_cd4_neg_Cycling"
[3] "B_cell_Cycling|cd3_pos_cd4_neg_Cycling"                  
[4] "B_cell_Plasmablasts|cd3_pos_cd4_neg_Cycling"             
[5] "B_cell_Atypical|cd3_pos_cd4_neg_CTLs"                    
[6] "B_cell_Age-associated/autoimmune|cd3_pos_cd4_neg_CTLs"   
[7] "B_cell_Cycling|cd3_pos_cd4_neg_CTLs"                     
[8] "B_cell_Plasmablasts|cd3_pos_cd4_neg_CTLs"

[1] TRUE

[1] "cd3_pos_cd8_neg_Tregs|B_cell_Atypical"                            
 [2] "cd3_pos_cd8_neg_Tregs|B_cell_Age-associated/autoimmune"           
 [3] "cd3_pos_cd8_neg_Tregs|B_cell_Cycling"                             
 [4] "cd3_pos_cd8_neg_Tregs|B_cell_Plasmablasts"                        
 [5] "cd3_pos_cd8_neg_Follicular Tregs|B_cell_Atypical"                 
 [6] "cd3_pos_cd8_neg_Follicular Tregs|B_cell_Age-associated/autoimmune"
 [7] "cd3_pos_cd8_neg_Follicular Tregs|B_cell_Cycling"                  
 [8] "cd3_pos_cd8_neg_Follicular Tregs|B_cell_Plasmablasts"             
 [9] "cd3_pos_cd8_neg_Cycling|B_cell_Atypical"                          
[10] "cd3_pos_cd8_neg_Cycling|B_cell_Age-associated/autoimmune"         
[11] "cd3_pos_cd8_neg_Cycling|B_cell_Cycling"                           
[12] "cd3_pos_cd8_neg_Cycling|B_cell_Plasmablasts"                      
[13] "cd3_pos_cd8_neg_CTLs|B_cell_Atypical"                             
[14] "cd3_pos_cd8_neg_CTLs|B_cell_Age-associated/autoimmune"            
[15] "cd3_pos_cd8_neg_CTLs|B_cell_Cycling"                              
[16] "cd3_pos_cd8_neg_CTLs|B_cell_Plasmablasts"

...

[1] TRUE

[1] "non_B_non_T_Macrophages|B_cell_Atypical"                 
 [2] "non_B_non_T_Macrophages|B_cell_Age-associated/autoimmune"
 [3] "non_B_non_T_Macrophages|B_cell_Cycling"                  
 [4] "non_B_non_T_Macrophages|B_cell_Plasmablasts"             
 [5] "non_B_non_T_NK cells|B_cell_Atypical"                    
 [6] "non_B_non_T_NK cells|B_cell_Age-associated/autoimmune"   
 [7] "non_B_non_T_NK cells|B_cell_Cycling"                     
 [8] "non_B_non_T_NK cells|B_cell_Plasmablasts"                
 [9] "non_B_non_T_Cycling|B_cell_Atypical"                     
[10] "non_B_non_T_Cycling|B_cell_Age-associated/autoimmune"    
[11] "non_B_non_T_Cycling|B_cell_Cycling"                      
[12] "non_B_non_T_Cycling|B_cell_Plasmablasts"                 
[13] "non_B_non_T_Neutrophils|B_cell_Atypical"                 
[14] "non_B_non_T_Neutrophils|B_cell_Age-associated/autoimmune"
[15] "non_B_non_T_Neutrophils|B_cell_Cycling"                  
[16] "non_B_non_T_Neutrophils|B_cell_Plasmablasts"

[1] TRUE

[1] "B_cell_Atypical|non_B_non_T_Macrophages"                 
 [2] "B_cell_Age-associated/autoimmune|non_B_non_T_Macrophages"
 [3] "B_cell_Cycling|non_B_non_T_Macrophages"                  
 [4] "B_cell_Plasmablasts|non_B_non_T_Macrophages"             
 [5] "B_cell_Atypical|non_B_non_T_NK cells"                    
 [6] "B_cell_Age-associated/autoimmune|non_B_non_T_NK cells"   
 [7] "B_cell_Cycling|non_B_non_T_NK cells"                     
 [8] "B_cell_Plasmablasts|non_B_non_T_NK cells"                
 [9] "B_cell_Atypical|non_B_non_T_Cycling"                     
[10] "B_cell_Age-associated/autoimmune|non_B_non_T_Cycling"    
[11] "B_cell_Cycling|non_B_non_T_Cycling"                      
[12] "B_cell_Plasmablasts|non_B_non_T_Cycling"                 
[13] "B_cell_Atypical|non_B_non_T_Neutrophils"                 
[14] "B_cell_Age-associated/autoimmune|non_B_non_T_Neutrophils"
[15] "B_cell_Cycling|non_B_non_T_Neutrophils"                  
[16] "B_cell_Plasmablasts|non_B_non_T_Neutrophils"

[1] TRUE

In [6]:
cd3_pos_cd4_neg_B_cell_mtx_bcl6 = build_cpdb_mtx_noSick(cols = cd3_pos_cd4_neg_B_cell_interacts,
                                            scaled = FALSE,
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1, 
                                            group_name = "_bcl6",
                                            ph_6mos_pvals = ph_bcl6_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_18mos_means)

B_cell_cd3_pos_cd4_neg_mtx_bcl6 = build_cpdb_mtx_noSick(cols = B_cell_cd3_pos_cd4_neg_interacts,
                                            scaled = FALSE,
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1, 
                                            group_name = "_bcl6",
                                            ph_6mos_pvals = ph_bcl6_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_18mos_means)

cd3_pos_cd8_neg_B_cell_mtx_bcl6 = build_cpdb_mtx_noSick(cols = cd3_pos_cd8_neg_B_cell_interacts,
                                            scaled = FALSE,
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1,  
                                            group_name = "_bcl6",
                                            ph_6mos_pvals = ph_bcl6_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_18mos_means)

B_cell_cd3_pos_cd8_neg_mtx_bcl6 = build_cpdb_mtx_noSick(cols = B_cell_cd3_pos_cd8_neg_interacts, 
                                            scaled = FALSE,
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1,  
                                            group_name = "_bcl6",
                                            ph_6mos_pvals = ph_bcl6_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_18mos_means)

non_B_non_T_B_cell_mtx_bcl6 = build_cpdb_mtx_noSick(cols = non_B_non_T_B_cell_interacts,
                                            scaled = FALSE,
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1, 
                                            group_name = "_bcl6",
                                            ph_6mos_pvals = ph_bcl6_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_18mos_means)

B_cell_non_B_non_T_mtx_bcl6 = build_cpdb_mtx_noSick(cols = B_cell_non_B_non_T_interacts,
                                            scaled = FALSE,
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1, 
                                            group_name = "_bcl6",
                                            ph_6mos_pvals = ph_bcl6_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_18mos_means)

total number of intersections considered: 43

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 50

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 57

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 57

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 73

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 79

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data



In [7]:
cd3_pos_cd4_neg_B_cell_mtx_bcl6_double = build_cpdb_mtx_noSick(cols = cd3_pos_cd4_neg_B_cell_interacts,
                                            scaled = FALSE,
                                            group_name = "_bcl6_double",
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_bcl6_double_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_double_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_double_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_double_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_double_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_double_18mos_means)

B_cell_cd3_pos_cd4_neg_mtx_bcl6_double = build_cpdb_mtx_noSick(cols = B_cell_cd3_pos_cd4_neg_interacts,
                                            scaled = FALSE,            
                                            group_name = "_bcl6_double",
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_bcl6_double_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_double_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_double_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_double_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_double_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_double_18mos_means)

cd3_pos_cd8_neg_B_cell_mtx_bcl6_double = build_cpdb_mtx_noSick(cols = cd3_pos_cd8_neg_B_cell_interacts,
                                            scaled = FALSE,
                                            group_name = "_bcl6_double",
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_bcl6_double_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_double_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_double_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_double_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_double_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_double_18mos_means)

B_cell_cd3_pos_cd8_neg_mtx_bcl6_double = build_cpdb_mtx_noSick(cols = B_cell_cd3_pos_cd8_neg_interacts, 
                                            scaled = FALSE,
                                            group_name = "_bcl6_double",
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_bcl6_double_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_double_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_double_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_double_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_double_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_double_18mos_means)

non_B_non_T_B_cell_mtx_bcl6_double = build_cpdb_mtx_noSick(cols = non_B_non_T_B_cell_interacts,
                                            scaled = FALSE,
                                            group_name = "_bcl6_double",
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_bcl6_double_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_double_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_double_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_double_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_double_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_double_18mos_means)

B_cell_non_B_non_T_mtx_bcl6_double = build_cpdb_mtx_noSick(cols = B_cell_non_B_non_T_interacts,
                                            scaled = FALSE,
                                            group_name = "_bcl6_double",
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_bcl6_double_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_double_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_double_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_double_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_double_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_double_18mos_means)

total number of intersections considered: 40

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 44

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 55

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 61

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 67

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 81

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data



In [8]:
cd3_pos_cd4_neg_B_cell_mtx_wt = build_cpdb_mtx_wt(cols = cd3_pos_cd4_neg_B_cell_interacts,
                                            scaled = FALSE,
                                            group_name = "_wt",
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_wt_6mos_pvals,
                                            ph_6mos_means = ph_wt_6mos_means,
                                            ph_14mos_pvals = ph_wt_14mos_pvals,
                                            ph_14mos_means = ph_wt_14mos_means,
                                            ph_18mos_pvals = ph_wt_18mos_pvals,
                                            ph_18mos_means = ph_wt_18mos_means)

B_cell_cd3_pos_cd4_neg_mtx_wt = build_cpdb_mtx_wt(cols = B_cell_cd3_pos_cd4_neg_interacts,
                                            scaled = FALSE,            
                                            group_name = "_wt",
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_wt_6mos_pvals,
                                            ph_6mos_means = ph_wt_6mos_means,
                                            ph_14mos_pvals = ph_wt_14mos_pvals,
                                            ph_14mos_means = ph_wt_14mos_means,
                                            ph_18mos_pvals = ph_wt_18mos_pvals,
                                            ph_18mos_means = ph_wt_18mos_means)

cd3_pos_cd8_neg_B_cell_mtx_wt = build_cpdb_mtx_wt(cols = cd3_pos_cd8_neg_B_cell_interacts,
                                            scaled = FALSE,
                                            group_name = "_wt",
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_wt_6mos_pvals,
                                            ph_6mos_means = ph_wt_6mos_means,
                                            ph_14mos_pvals = ph_wt_14mos_pvals,
                                            ph_14mos_means = ph_wt_14mos_means,
                                            ph_18mos_pvals = ph_wt_18mos_pvals,
                                            ph_18mos_means = ph_wt_18mos_means)

B_cell_cd3_pos_cd8_neg_mtx_wt = build_cpdb_mtx_wt(cols = B_cell_cd3_pos_cd8_neg_interacts, 
                                            scaled = FALSE,
                                            group_name = "_wt",
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_wt_6mos_pvals,
                                            ph_6mos_means = ph_wt_6mos_means,
                                            ph_14mos_pvals = ph_wt_14mos_pvals,
                                            ph_14mos_means = ph_wt_14mos_means,
                                            ph_18mos_pvals = ph_wt_18mos_pvals,
                                            ph_18mos_means = ph_wt_18mos_means)

non_B_non_T_B_cell_mtx_wt = build_cpdb_mtx_wt(cols = non_B_non_T_B_cell_interacts,
                                            scaled = FALSE,
                                            group_name = "_wt",
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_wt_6mos_pvals,
                                            ph_6mos_means = ph_wt_6mos_means,
                                            ph_14mos_pvals = ph_wt_14mos_pvals,
                                            ph_14mos_means = ph_wt_14mos_means,
                                            ph_18mos_pvals = ph_wt_18mos_pvals,
                                            ph_18mos_means = ph_wt_18mos_means)

B_cell_non_B_non_T_mtx_wt = build_cpdb_mtx_wt(cols = B_cell_non_B_non_T_interacts,
                                            scaled = FALSE,
                                            group_name = "_wt",
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_wt_6mos_pvals,
                                            ph_6mos_means = ph_wt_6mos_means,
                                            ph_14mos_pvals = ph_wt_14mos_pvals,
                                            ph_14mos_means = ph_wt_14mos_means,
                                            ph_18mos_pvals = ph_wt_18mos_pvals,
                                            ph_18mos_means = ph_wt_18mos_means)

total number of intersections considered: 37

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 34

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 59

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 51

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 62

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 69

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data



In [9]:
B_cell_cd3_pos_cd4_neg_mtx<- merge_mtx(B_cell_cd3_pos_cd4_neg_mtx_wt, B_cell_cd3_pos_cd4_neg_mtx_bcl6, B_cell_cd3_pos_cd4_neg_mtx_bcl6_double)
cd3_pos_cd4_neg_B_cell_mtx<- merge_mtx(cd3_pos_cd4_neg_B_cell_mtx_wt, cd3_pos_cd4_neg_B_cell_mtx_bcl6, cd3_pos_cd4_neg_B_cell_mtx_bcl6_double)
cd3_pos_cd8_neg_B_cell_mtx<- merge_mtx(cd3_pos_cd8_neg_B_cell_mtx_wt, cd3_pos_cd8_neg_B_cell_mtx_bcl6, cd3_pos_cd8_neg_B_cell_mtx_bcl6_double)
B_cell_cd3_pos_cd8_neg_mtx<- merge_mtx(B_cell_cd3_pos_cd8_neg_mtx_wt, B_cell_cd3_pos_cd8_neg_mtx_bcl6, B_cell_cd3_pos_cd8_neg_mtx_bcl6_double)
non_B_non_T_B_cell_mtx<- merge_mtx(non_B_non_T_B_cell_mtx_wt, non_B_non_T_B_cell_mtx_bcl6, non_B_non_T_B_cell_mtx_bcl6_double)
B_cell_non_B_non_T_mtx<- merge_mtx(B_cell_non_B_non_T_mtx_wt, B_cell_non_B_non_T_mtx_bcl6, B_cell_non_B_non_T_mtx_bcl6_double)

wt...

bcl6...

bcl6 double...



[1] NA NA


TRUE

scaling the count matrix

wt...

bcl6...

bcl6 double...



[1] NA NA


TRUE

scaling the count matrix

wt...

bcl6...

bcl6 double...



...

TRUE

scaling the count matrix

wt...

bcl6...

bcl6 double...



[1] NA NA


TRUE

scaling the count matrix

wt...

bcl6...

bcl6 double...



[1] NA NA


TRUE

scaling the count matrix



In [10]:
hp1 = plt_cpdb_dotplot(exp_mtx = cd3_pos_cd4_neg_B_cell_mtx$exp_mtx, 
                       pval_mtx = cd3_pos_cd4_neg_B_cell_mtx$pval_mtx)
hp2 = plt_cpdb_dotplot(exp_mtx = B_cell_cd3_pos_cd4_neg_mtx$exp_mtx, 
                       pval_mtx = B_cell_cd3_pos_cd4_neg_mtx$pval_mtx)

hp3 = plt_cpdb_dotplot(exp_mtx = cd3_pos_cd8_neg_B_cell_mtx$exp_mtx, 
                       pval_mtx = cd3_pos_cd8_neg_B_cell_mtx$pval_mtx)
hp4 = plt_cpdb_dotplot(exp_mtx = B_cell_cd3_pos_cd8_neg_mtx$exp_mtx, 
                       pval_mtx = B_cell_cd3_pos_cd8_neg_mtx$pval_mtx)

hp5 = plt_cpdb_dotplot(exp_mtx = non_B_non_T_B_cell_mtx$exp_mtx, 
                       pval_mtx = non_B_non_T_B_cell_mtx$pval_mtx)

hp6 = plt_cpdb_dotplot(exp_mtx = B_cell_non_B_non_T_mtx$exp_mtx, 
                       pval_mtx = B_cell_non_B_non_T_mtx$pval_mtx)

quantile: -1.02568803899899-0.3030335929096032.02713324952887



genotype
       bcl6 bcl6_double          wt 
         24          24          24 
age
06mos 14mos 18mos 
   24    24    24 


quantile: -1.0114373317163-0.2699414299742481.82002626915613



genotype
       bcl6 bcl6_double          wt 
         24          24          24 
age
06mos 14mos 18mos 
   24    24    24 


quantile: -1.00433280491095-0.3096263534901981.97975820031048



...

genotype
       bcl6 bcl6_double          wt 
         48          48          48 
age
06mos 14mos 18mos 
   48    48    48 


quantile: -0.964996104106395-0.3767683762323031.9393665901289



genotype
       bcl6 bcl6_double          wt 
         48          48          48 
age
06mos 14mos 18mos 
   48    48    48 


quantile: -1.05823977548783-0.2915225257341511.90054094070601



genotype
       bcl6 bcl6_double          wt 
         48          48          48 
age
06mos 14mos 18mos 
   48    48    48 


In [12]:
legend_dot_size = unit(1.5,"mm")
lgd = Legend(labels = c(0, 1, 2, 3, 4, 5),
         title = "-logPvalue",
         type = "points",
         graphics = list(
            function(x, y, w, h) grid.circle(x, y, r= sqrt(0/5) * legend_dot_size,
                                            gp = gpar(fill = "black")),
            function(x, y, w, h) grid.circle(x, y, r= sqrt(1/5) * legend_dot_size,
                                            gp = gpar(fill = "black")),
            function(x, y, w, h) grid.circle(x, y, r= sqrt(2/5) * legend_dot_size,
                                            gp = gpar(fill = "black")),
            function(x, y, w, h) grid.circle(x, y, r= sqrt(3/5) * legend_dot_size,
                                            gp = gpar(fill = "black")),
            function(x, y, w, h) grid.circle(x, y, r= sqrt(4/5) * legend_dot_size,
                                            gp = gpar(fill = "black")),
            function(x, y, w, h) grid.circle(x, y, r= sqrt(5/5) * legend_dot_size,
                                            gp = gpar(fill = "black"))            
                    )
             )

In [13]:
pdf("./results/figures/9_Dotplot_cd3_pos_cd4_neg_B_cell_wt_bcl6_and_bcl6_double_cell3_time1_genotype1.pdf", width = 26, height = 15)
draw(hp1, annotation_legend_list = lgd, ht_gap = unit(1, "cm"))
dev.off()

pdf("./results/figures/9_Dotplot_B_cell_cd3_pos_cd4_neg_wt_bcl6_and_bcl6_double_cell3_time1_genotype1.pdf", width = 26, height = 15)
draw(hp2, annotation_legend_list = lgd, ht_gap = unit(1, "cm"))
dev.off()

pdf("./results/figures/9_Dotplot_cd3_pos_cd8_neg_B_cell_wt_bcl6_and_bcl6_double_cell3_time1_genotype1.pdf", width = 26, height = 15)
draw(hp3, annotation_legend_list = lgd, ht_gap = unit(1, "cm"))
dev.off()

pdf("./results/figures/9_Dotplot_B_cell_cd3_pos_cd8_neg_wt_bcl6_and_bcl6_double_cell3_time1_genotype1.pdf", width = 26, height = 15)
draw(hp4, annotation_legend_list = lgd, ht_gap = unit(1, "cm"))
dev.off()

pdf("./results/figures/9_Dotplot_non_B_non_T_B_cell_wt_bcl6_and_bcl6_double_cell3_time1_genotype1.pdf", width = 26, height = 15)
draw(hp5, annotation_legend_list = lgd, ht_gap = unit(1, "cm"))
dev.off()

pdf("./results/figures/9_Dotplot_B_cell_non_B_non_T_wt_bcl6_and_bcl6_double_cell3_time1_genotype1.pdf", width = 26, height = 15)
draw(hp6, annotation_legend_list = lgd, ht_gap = unit(1, "cm"))
dev.off()

png 
  2

png 
  2

png 
  2

png 
  2

png 
  2

png 
  2

### 2. Between non_B non_T and Cd3_pos

In [14]:
non_B_non_T_clusters<- c("non_B_non_T_Macrophages", "non_B_non_T_NK cells", 
                             "non_B_non_T_Cycling", "non_B_non_T_Neutrophils")
non_B_non_T_clusters_otherThree<- non_B_non_T_clusters[c(4,1,3)]
non_B_non_T_clusters_NK<- non_B_non_T_clusters[2]

cd3_pos_cd4_neg_clusters<- c("cd3_pos_cd4_neg_Cycling", "cd3_pos_cd4_neg_CTLs")
cd3_pos_cd8_neg_clusters<- c("cd3_pos_cd8_neg_Tregs", "cd3_pos_cd8_neg_Follicular Tregs", 
                             "cd3_pos_cd8_neg_Cycling", "cd3_pos_cd8_neg_CTLs")

# neutrophils, macrophages, cycling VS cd3_pos_cd4_neg

non_B_non_T_clusters_otherThree_cd3_pos_cd4_neg_interacts<- c()
cd3_pos_cd4_neg_non_B_non_T_clusters_otherThree_interacts<- c()


for (c1 in non_B_non_T_clusters_otherThree){
    for (c2 in cd3_pos_cd4_neg_clusters){
        
        interact<- paste0(c1, "|", c2)
        inverse_interact<- paste0(c2, "|", c1)
        
        non_B_non_T_clusters_otherThree_cd3_pos_cd4_neg_interacts<- c(non_B_non_T_clusters_otherThree_cd3_pos_cd4_neg_interacts, interact)
        cd3_pos_cd4_neg_non_B_non_T_clusters_otherThree_interacts<- c(cd3_pos_cd4_neg_non_B_non_T_clusters_otherThree_interacts, inverse_interact)
    }
}

non_B_non_T_clusters_otherThree_cd3_pos_cd4_neg_interacts
sum(colnames(ph_wt_6mos_pvals) %in% non_B_non_T_clusters_otherThree_cd3_pos_cd4_neg_interacts) == length(non_B_non_T_clusters_otherThree_cd3_pos_cd4_neg_interacts)

cd3_pos_cd4_neg_non_B_non_T_clusters_otherThree_interacts
sum(colnames(ph_wt_6mos_pvals) %in% cd3_pos_cd4_neg_non_B_non_T_clusters_otherThree_interacts) == length(cd3_pos_cd4_neg_non_B_non_T_clusters_otherThree_interacts)

# neutrophils, macrophages, cycling VS cd3_pos_cd8_neg

non_B_non_T_clusters_otherThree_cd3_pos_cd8_neg_interacts<- c()
cd3_pos_cd8_neg_non_B_non_T_clusters_otherThree_interacts<- c()


for (c1 in non_B_non_T_clusters_otherThree){
    for (c2 in cd3_pos_cd8_neg_clusters){
        
        interact<- paste0(c1, "|", c2)
        inverse_interact<- paste0(c2, "|", c1)
        
        non_B_non_T_clusters_otherThree_cd3_pos_cd8_neg_interacts<- c(non_B_non_T_clusters_otherThree_cd3_pos_cd8_neg_interacts, interact)
        cd3_pos_cd8_neg_non_B_non_T_clusters_otherThree_interacts<- c(cd3_pos_cd8_neg_non_B_non_T_clusters_otherThree_interacts, inverse_interact)
    }
}

non_B_non_T_clusters_otherThree_cd3_pos_cd8_neg_interacts
sum(colnames(ph_wt_6mos_pvals) %in% non_B_non_T_clusters_otherThree_cd3_pos_cd8_neg_interacts) == length(non_B_non_T_clusters_otherThree_cd3_pos_cd8_neg_interacts)

cd3_pos_cd8_neg_non_B_non_T_clusters_otherThree_interacts
sum(colnames(ph_wt_6mos_pvals) %in% cd3_pos_cd8_neg_non_B_non_T_clusters_otherThree_interacts) == length(cd3_pos_cd8_neg_non_B_non_T_clusters_otherThree_interacts)


[1] "non_B_non_T_Neutrophils|cd3_pos_cd4_neg_Cycling"
[2] "non_B_non_T_Neutrophils|cd3_pos_cd4_neg_CTLs"   
[3] "non_B_non_T_Macrophages|cd3_pos_cd4_neg_Cycling"
[4] "non_B_non_T_Macrophages|cd3_pos_cd4_neg_CTLs"   
[5] "non_B_non_T_Cycling|cd3_pos_cd4_neg_Cycling"    
[6] "non_B_non_T_Cycling|cd3_pos_cd4_neg_CTLs"

[1] TRUE

[1] "cd3_pos_cd4_neg_Cycling|non_B_non_T_Neutrophils"
[2] "cd3_pos_cd4_neg_CTLs|non_B_non_T_Neutrophils"   
[3] "cd3_pos_cd4_neg_Cycling|non_B_non_T_Macrophages"
[4] "cd3_pos_cd4_neg_CTLs|non_B_non_T_Macrophages"   
[5] "cd3_pos_cd4_neg_Cycling|non_B_non_T_Cycling"    
[6] "cd3_pos_cd4_neg_CTLs|non_B_non_T_Cycling"

[1] TRUE

[1] "non_B_non_T_Neutrophils|cd3_pos_cd8_neg_Tregs"           
 [2] "non_B_non_T_Neutrophils|cd3_pos_cd8_neg_Follicular Tregs"
 [3] "non_B_non_T_Neutrophils|cd3_pos_cd8_neg_Cycling"         
 [4] "non_B_non_T_Neutrophils|cd3_pos_cd8_neg_CTLs"            
 [5] "non_B_non_T_Macrophages|cd3_pos_cd8_neg_Tregs"           
 [6] "non_B_non_T_Macrophages|cd3_pos_cd8_neg_Follicular Tregs"
 [7] "non_B_non_T_Macrophages|cd3_pos_cd8_neg_Cycling"         
 [8] "non_B_non_T_Macrophages|cd3_pos_cd8_neg_CTLs"            
 [9] "non_B_non_T_Cycling|cd3_pos_cd8_neg_Tregs"               
[10] "non_B_non_T_Cycling|cd3_pos_cd8_neg_Follicular Tregs"    
[11] "non_B_non_T_Cycling|cd3_pos_cd8_neg_Cycling"             
[12] "non_B_non_T_Cycling|cd3_pos_cd8_neg_CTLs"

[1] TRUE

[1] "cd3_pos_cd8_neg_Tregs|non_B_non_T_Neutrophils"           
 [2] "cd3_pos_cd8_neg_Follicular Tregs|non_B_non_T_Neutrophils"
 [3] "cd3_pos_cd8_neg_Cycling|non_B_non_T_Neutrophils"         
 [4] "cd3_pos_cd8_neg_CTLs|non_B_non_T_Neutrophils"            
 [5] "cd3_pos_cd8_neg_Tregs|non_B_non_T_Macrophages"           
 [6] "cd3_pos_cd8_neg_Follicular Tregs|non_B_non_T_Macrophages"
 [7] "cd3_pos_cd8_neg_Cycling|non_B_non_T_Macrophages"         
 [8] "cd3_pos_cd8_neg_CTLs|non_B_non_T_Macrophages"            
 [9] "cd3_pos_cd8_neg_Tregs|non_B_non_T_Cycling"               
[10] "cd3_pos_cd8_neg_Follicular Tregs|non_B_non_T_Cycling"    
[11] "cd3_pos_cd8_neg_Cycling|non_B_non_T_Cycling"             
[12] "cd3_pos_cd8_neg_CTLs|non_B_non_T_Cycling"

[1] TRUE

In [15]:
non_B_non_T_cd3_pos_cd4_neg_mtx_bcl6 = build_cpdb_mtx(cols = non_B_non_T_clusters_otherThree_cd3_pos_cd4_neg_interacts,
                                            scaled = FALSE,
                                            sig.cellpairs = 1,
                                            sig.timepoint = 1, 
                                            group_name = "_bcl6",
                                            ph_6mos_pvals = ph_bcl6_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_18mos_means,
                                            ph_sick_pvals = ph_bcl6_sick_pvals,
                                            ph_sick_means = ph_bcl6_sick_means)

cd3_pos_cd4_neg_non_B_non_T_mtx_bcl6 = build_cpdb_mtx(cols = cd3_pos_cd4_neg_non_B_non_T_clusters_otherThree_interacts,
                                            scaled = FALSE,
                                            sig.cellpairs = 1,
                                            sig.timepoint = 1, 
                                            group_name = "_bcl6",
                                            ph_6mos_pvals = ph_bcl6_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_18mos_means,
                                            ph_sick_pvals = ph_bcl6_sick_pvals,
                                            ph_sick_means = ph_bcl6_sick_means)


non_B_non_T_cd3_pos_cd4_neg_mtx_bcl6_double = build_cpdb_mtx(cols = non_B_non_T_clusters_otherThree_cd3_pos_cd4_neg_interacts,
                                            scaled = FALSE,
                                            group_name = "_bcl6_double",
                                            sig.cellpairs = 1,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_bcl6_double_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_double_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_double_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_double_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_double_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_double_18mos_means,
                                            ph_sick_pvals = ph_bcl6_double_sick_pvals,
                                            ph_sick_means = ph_bcl6_double_sick_means)

cd3_pos_cd4_neg_non_B_non_T_mtx_bcl6_double = build_cpdb_mtx(cols = cd3_pos_cd4_neg_non_B_non_T_clusters_otherThree_interacts,
                                            scaled = FALSE,
                                            group_name = "_bcl6_double",
                                            sig.cellpairs = 1,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_bcl6_double_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_double_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_double_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_double_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_double_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_double_18mos_means,
                                            ph_sick_pvals = ph_bcl6_double_sick_pvals,
                                            ph_sick_means = ph_bcl6_double_sick_means)

non_B_non_T_cd3_pos_cd4_neg_mtx_wt = build_cpdb_mtx_wt(cols = non_B_non_T_clusters_otherThree_cd3_pos_cd4_neg_interacts,
                                            scaled = FALSE,
                                            group_name = "_wt",
                                            sig.cellpairs = 1,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_wt_6mos_pvals,
                                            ph_6mos_means = ph_wt_6mos_means,
                                            ph_14mos_pvals = ph_wt_14mos_pvals,
                                            ph_14mos_means = ph_wt_14mos_means,
                                            ph_18mos_pvals = ph_wt_18mos_pvals,
                                            ph_18mos_means = ph_wt_18mos_means)

cd3_pos_cd4_neg_non_B_non_T_mtx_wt = build_cpdb_mtx_wt(cols = cd3_pos_cd4_neg_non_B_non_T_clusters_otherThree_interacts,
                                            scaled = FALSE,
                                            group_name = "_wt",
                                            sig.cellpairs = 1,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_wt_6mos_pvals,
                                            ph_6mos_means = ph_wt_6mos_means,
                                            ph_14mos_pvals = ph_wt_14mos_pvals,
                                            ph_14mos_means = ph_wt_14mos_means,
                                            ph_18mos_pvals = ph_wt_18mos_pvals,
                                            ph_18mos_means = ph_wt_18mos_means)

total number of intersections considered: 93

6 mos...

14 mos...

18 mos...

sick...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 92

6 mos...

14 mos...

18 mos...

sick...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 78

6 mos...

14 mos...

18 mos...

sick...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 83

6 mos...

14 mos...

18 mos...

sick...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 83

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 76

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data



In [38]:
plt_cpdb_dotplot<- function(exp_mtx, pval_mtx, dt.size = 1.5, 
                            column_title = "merged"){
    
    col_len = ncol(exp_mtx)/11
    
    message("quantile: ", c(quantile(exp_mtx, 0.1), quantile(exp_mtx, 0.5), quantile(exp_mtx, 0.95)))

    col_fun = circlize::colorRamp2(c(-1.5, 0, 1.5), c("#440154FF", "#238A8DFF", "#FDE725FF"))

    fact = gsub("06mos.*|14mos.*|18mos.*|sick.*", "", colnames(exp_mtx))
    dend = cluster_between_groups(exp_mtx, fact)
    
    legend_dot_size = unit(dt.size,"mm")
    
    lgd = Legend(labels = c(0, 1, 2, 3, 4, 5),
                 title = "-logPvalue",
                 type = "points",
                 graphics = list(
                    function(x, y, w, h) grid.circle(x, y, r= sqrt(0/5) * legend_dot_size,
                                                    gp = gpar(fill = "black")),
                    function(x, y, w, h) grid.circle(x, y, r= sqrt(1/5) * legend_dot_size,
                                                    gp = gpar(fill = "black")),
                    function(x, y, w, h) grid.circle(x, y, r= sqrt(2/5) * legend_dot_size,
                                                    gp = gpar(fill = "black")),
                    function(x, y, w, h) grid.circle(x, y, r= sqrt(3/5) * legend_dot_size,
                                                    gp = gpar(fill = "black")),
                    function(x, y, w, h) grid.circle(x, y, r= sqrt(4/5) * legend_dot_size,
                                                    gp = gpar(fill = "black")),
                    function(x, y, w, h) grid.circle(x, y, r= sqrt(5/5) * legend_dot_size,
                                                    gp = gpar(fill = "black"))            
                            )
                     )

    layer_fun_dot = function(j, i, x, y, w, h, fill){
        grid.rect(x = x, y = y, width = w, height = h, 
                  gp = gpar(col = NA, fill = NA))

        grid.circle(x=x, y=y, r=sqrt(pindex(pval_mtx, i, j)/5) * legend_dot_size,
                    gp = gpar(fill = col_fun(pindex(exp_mtx, i, j)), col = NA))
    }
    
    clusterID =  gsub("_06mos.*|_14mos.*|_18mos.*|_sick.*", "", colnames(exp_mtx))
    genotype = paste0("bcl", gsub(pattern = ".*bcl|.*wt", replacement = "", x = colnames(exp_mtx)))
    genotype[genotype=='bcl'] = 'wt'
    age = rep(rep(c("06mos", "14mos", "18mos", "sick"), times = c(3,3,3,2)), ncol(exp_mtx)/11)
    
    message(table(genotype))
    message(table(age))
    anno_df = data.frame(clusterID = clusterID, genotype = genotype, age = age)
    
    # clusterID_cols
    clusterID_cols= kelly(n = length(unique(clusterID)) +2)[-c(1,2)] %>% unname()
    clusterID_cols = setNames(clusterID_cols, unique(clusterID))
    # age_cols
    age_cols = brewer.pal(4, "Dark2")
    age_cols = setNames(age_cols, c("06mos", "14mos", "18mos", "sick"))
    
    genotype_cols = brewer.pal(5, "Set1")[c(1,3,5)]
    genotype_cols = setNames(genotype_cols, c("wt", "bcl6", "bcl6_double"))
    
    col_list = list(clusterID = clusterID_cols, genotype = genotype_cols, age = age_cols)

    column_ha <- HeatmapAnnotation(
        clusterID = clusterID, age = age, genotype = genotype, col = col_list,
        annotation_legend_param = list(clusterID = list(title = "annotation"),
                                       age = list(title = "age"),
                                       genotype = list(title = "genotype")), show_annotation_name = TRUE)
    
    dotplot<- Heatmap(exp_mtx, 
        name = "meanExpr",
        column_title = column_title,
        column_names_side = "top",
        column_title_side = "bottom",
        show_column_names = FALSE,
        column_names_gp = gpar(fontsize = 10),
        column_title_gp = gpar(fontsize = 12),
        cluster_columns = dend,
        column_split = col_len,
        rect_gp = gpar(type = "none"),
        layer_fun = layer_fun_dot,
        col = col_fun,
        row_km = 6,
        row_names_gp = gpar(fontsize = 8),
        top_annotation = column_ha,
        border = "black",
        show_row_dend =FALSE,
        show_column_dend = FALSE)
        
    return(dotplot)
    
}

legend_dot_size = unit(1.5,"mm")
lgd = Legend(labels = c(0, 1, 2, 3, 4, 5),
         title = "-logPvalue",
         type = "points",
         graphics = list(
            function(x, y, w, h) grid.circle(x, y, r= sqrt(0/5) * legend_dot_size,
                                            gp = gpar(fill = "black")),
            function(x, y, w, h) grid.circle(x, y, r= sqrt(1/5) * legend_dot_size,
                                            gp = gpar(fill = "black")),
            function(x, y, w, h) grid.circle(x, y, r= sqrt(2/5) * legend_dot_size,
                                            gp = gpar(fill = "black")),
            function(x, y, w, h) grid.circle(x, y, r= sqrt(3/5) * legend_dot_size,
                                            gp = gpar(fill = "black")),
            function(x, y, w, h) grid.circle(x, y, r= sqrt(4/5) * legend_dot_size,
                                            gp = gpar(fill = "black")),
            function(x, y, w, h) grid.circle(x, y, r= sqrt(5/5) * legend_dot_size,
                                            gp = gpar(fill = "black"))            
                    )
             )

In [39]:
non_B_non_T_cd3_pos_cd4_neg_mtx<- merge_mtx(non_B_non_T_cd3_pos_cd4_neg_mtx_wt, non_B_non_T_cd3_pos_cd4_neg_mtx_bcl6, non_B_non_T_cd3_pos_cd4_neg_mtx_bcl6_double)
cd3_pos_cd4_neg_non_B_non_T_mtx<- merge_mtx(cd3_pos_cd4_neg_non_B_non_T_mtx_wt, cd3_pos_cd4_neg_non_B_non_T_mtx_bcl6, cd3_pos_cd4_neg_non_B_non_T_mtx_bcl6_double)

hp7 = plt_cpdb_dotplot(exp_mtx = non_B_non_T_cd3_pos_cd4_neg_mtx$exp_mtx, 
                       pval_mtx = non_B_non_T_cd3_pos_cd4_neg_mtx$pval_mtx)

hp8 = plt_cpdb_dotplot(exp_mtx = cd3_pos_cd4_neg_non_B_non_T_mtx$exp_mtx, 
                       pval_mtx = cd3_pos_cd4_neg_non_B_non_T_mtx$pval_mtx)

pdf("./results/figures/9_Dotplot_non_B_non_T_cd3_pos_cd4_neg_wt_bcl6_and_bcl6_double_cell1_time1_genotype1.pdf", width = 15, height = 12)
draw(hp7, annotation_legend_list = lgd, ht_gap = unit(1, "cm"))
dev.off()

pdf("./results/figures/9_Dotplot_cd3_pos_cd4_neg_non_B_non_T_wt_bcl6_and_bcl6_double_cell1_time1_genotype1.pdf", width = 15, height = 12)
draw(hp8, annotation_legend_list = lgd, ht_gap = unit(1, "cm"))
dev.off()

wt...

bcl6...

bcl6 double...



[1] NA NA


TRUE

scaling the count matrix

wt...

bcl6...

bcl6 double...



[1] NA NA


TRUE

scaling the count matrix

quantile: -0.942737492906109-0.2738629066512381.84663170051678

242418

18181812

quantile: -0.966148648524009-0.3139667595125751.9995816117058

242418

18181812



png 
  2

png 
  2

In [24]:
non_B_non_T_cd3_pos_cd8_neg_mtx_bcl6 = build_cpdb_mtx(cols = non_B_non_T_clusters_otherThree_cd3_pos_cd8_neg_interacts,
                                            scaled = FALSE,
                                            sig.cellpairs = 1,
                                            sig.timepoint = 1, 
                                            group_name = "_bcl6",
                                            ph_6mos_pvals = ph_bcl6_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_18mos_means,
                                            ph_sick_pvals = ph_bcl6_sick_pvals,
                                            ph_sick_means = ph_bcl6_sick_means)

cd3_pos_cd8_neg_non_B_non_T_mtx_bcl6 = build_cpdb_mtx(cols = cd3_pos_cd8_neg_non_B_non_T_clusters_otherThree_interacts,
                                            scaled = FALSE,
                                            sig.cellpairs = 1,
                                            sig.timepoint = 1, 
                                            group_name = "_bcl6",
                                            ph_6mos_pvals = ph_bcl6_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_18mos_means,
                                            ph_sick_pvals = ph_bcl6_sick_pvals,
                                            ph_sick_means = ph_bcl6_sick_means)


non_B_non_T_cd3_pos_cd8_neg_mtx_bcl6_double = build_cpdb_mtx(cols = non_B_non_T_clusters_otherThree_cd3_pos_cd8_neg_interacts,
                                            scaled = FALSE,
                                            group_name = "_bcl6_double",
                                            sig.cellpairs = 1,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_bcl6_double_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_double_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_double_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_double_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_double_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_double_18mos_means,
                                            ph_sick_pvals = ph_bcl6_double_sick_pvals,
                                            ph_sick_means = ph_bcl6_double_sick_means)

cd3_pos_cd8_neg_non_B_non_T_mtx_bcl6_double = build_cpdb_mtx(cols = cd3_pos_cd8_neg_non_B_non_T_clusters_otherThree_interacts,
                                            scaled = FALSE,
                                            group_name = "_bcl6_double",
                                            sig.cellpairs = 1,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_bcl6_double_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_double_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_double_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_double_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_double_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_double_18mos_means,
                                            ph_sick_pvals = ph_bcl6_double_sick_pvals,
                                            ph_sick_means = ph_bcl6_double_sick_means)

non_B_non_T_cd3_pos_cd8_neg_mtx_wt = build_cpdb_mtx_wt(cols = non_B_non_T_clusters_otherThree_cd3_pos_cd8_neg_interacts,
                                            scaled = FALSE,
                                            group_name = "_wt",
                                            sig.cellpairs = 1,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_wt_6mos_pvals,
                                            ph_6mos_means = ph_wt_6mos_means,
                                            ph_14mos_pvals = ph_wt_14mos_pvals,
                                            ph_14mos_means = ph_wt_14mos_means,
                                            ph_18mos_pvals = ph_wt_18mos_pvals,
                                            ph_18mos_means = ph_wt_18mos_means)

cd3_pos_cd8_neg_non_B_non_T_mtx_wt = build_cpdb_mtx_wt(cols = cd3_pos_cd8_neg_non_B_non_T_clusters_otherThree_interacts,
                                            scaled = FALSE,
                                            group_name = "_wt",
                                            sig.cellpairs = 1,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_wt_6mos_pvals,
                                            ph_6mos_means = ph_wt_6mos_means,
                                            ph_14mos_pvals = ph_wt_14mos_pvals,
                                            ph_14mos_means = ph_wt_14mos_means,
                                            ph_18mos_pvals = ph_wt_18mos_pvals,
                                            ph_18mos_means = ph_wt_18mos_means)

total number of intersections considered: 109

6 mos...

14 mos...

18 mos...

sick...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 117

6 mos...

14 mos...

18 mos...

sick...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 105

6 mos...

14 mos...

18 mos...

sick...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 118

6 mos...

14 mos...

18 mos...

sick...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 88

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 91

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data



In [25]:
non_B_non_T_cd3_pos_cd8_neg_mtx<- merge_mtx(non_B_non_T_cd3_pos_cd8_neg_mtx_wt, non_B_non_T_cd3_pos_cd8_neg_mtx_bcl6, non_B_non_T_cd3_pos_cd8_neg_mtx_bcl6_double)
cd3_pos_cd8_neg_non_B_non_T_mtx<- merge_mtx(cd3_pos_cd8_neg_non_B_non_T_mtx_wt, cd3_pos_cd8_neg_non_B_non_T_mtx_bcl6, cd3_pos_cd8_neg_non_B_non_T_mtx_bcl6_double)

hp7 = plt_cpdb_dotplot(exp_mtx = non_B_non_T_cd3_pos_cd8_neg_mtx$exp_mtx, 
                       pval_mtx = non_B_non_T_cd3_pos_cd8_neg_mtx$pval_mtx)

hp8 = plt_cpdb_dotplot(exp_mtx = cd3_pos_cd8_neg_non_B_non_T_mtx$exp_mtx, 
                       pval_mtx = cd3_pos_cd8_neg_non_B_non_T_mtx$pval_mtx)

pdf("./results/figures/9_Dotplot_non_B_non_T_cd3_pos_cd8_neg_wt_bcl6_and_bcl6_double_cell1_time1_genotype1.pdf", width = 24, height = 15)
draw(hp7, annotation_legend_list = lgd, ht_gap = unit(1, "cm"))
dev.off()

pdf("./results/figures/9_Dotplot_cd3_pos_cd8_neg_non_B_non_T_wt_bcl6_and_bcl6_double_cell1_time1_genotype1.pdf", width = 24, height = 15)
draw(hp8, annotation_legend_list = lgd, ht_gap = unit(1, "cm"))
dev.off()

wt...

bcl6...

bcl6 double...



[1] NA NA


TRUE

scaling the count matrix

wt...

bcl6...

bcl6 double...



[1] NA NA


TRUE

scaling the count matrix

quantile: -0.971613342004948-0.3291149792447641.93609449537178

484836

36363624

quantile: -0.896750945131094-0.3310627443220371.95998004282277

484836

36363624



png 
  2

png 
  2

### 3. Interesting interactions between cd3_pos_cd4_neg CTLs/cycling with B-cell Malignant

In [41]:
cd3_pos_cd4_neg_B_cell_interactions<- c("IL10_receptor_IL10", "CTLA4_CD80", "CTLA4_CD86", "PDCD1_PDCD1LG2", 
                                        "PDCD1_FAM3C", "PDCD1_CD274", "CCL4_CNR2", "IFNG_Type_II_IFNR", "CCL5_CCR1",
                                        "SEMA4A_PLXND1")

cd3_pos_cd4_neg_B_cell_exp_mtx_sub<- cd3_pos_cd4_neg_B_cell_mtx$exp_mtx[cd3_pos_cd4_neg_B_cell_interactions, ]
cd3_pos_cd4_neg_B_cell_pval_mtx_sub<- cd3_pos_cd4_neg_B_cell_mtx$pval_mtx[cd3_pos_cd4_neg_B_cell_interactions, ]

cd3_pos_cd4_neg_B_cell_mtx_sub = list("exp_mtx" = cd3_pos_cd4_neg_B_cell_exp_mtx_sub, "pval_mtx" = cd3_pos_cd4_neg_B_cell_pval_mtx_sub)

hp1_sub = plt_cpdb_dotplot(exp_mtx = cd3_pos_cd4_neg_B_cell_mtx_sub$exp_mtx, 
                       pval_mtx = cd3_pos_cd4_neg_B_cell_mtx_sub$pval_mtx)

pdf("./results/figures/9_Dotplot_cd3_pos_cd4_neg_B_cell_wt_bcl6_and_bcl6_double_cell3_time1_genotype1_subset.pdf", width = 16, height = 4)
draw(hp1_sub, annotation_legend_list = lgd, ht_gap = unit(1, "cm"))
dev.off()

quantile: -1.08902079938769-0.1012412813361741.9536749522865



genotype
       bcl6 bcl6_double          wt 
         24          24          24 
age
06mos 14mos 18mos 
   24    24    24 


png 
  2

### 4. Interesting interactions between cd3_pos_cd8_neg CTLs and B-cell Malignant

In [42]:
cd3_pos_cd8_neg_CTLs_clusters<- c("cd3_pos_cd8_neg_CTLs")
B_cell_clusters<- c("B_cell_Atypical", "B_cell_Age-associated/autoimmune", 
                    "B_cell_Cycling", "B_cell_Plasmablasts")

cd3_pos_cd8_neg_CTLs_B_cell_interacts<- c()
B_cell_cd3_pos_cd8_neg_CTLs_interacts<- c()

for (c1 in cd3_pos_cd8_neg_CTLs_clusters){
    for (c2 in B_cell_clusters){
        
        interact<- paste0(c1, "|", c2)
        inverse_interact<- paste0(c2, "|", c1)
        
        cd3_pos_cd8_neg_CTLs_B_cell_interacts<- c(cd3_pos_cd8_neg_CTLs_B_cell_interacts, interact)
        B_cell_cd3_pos_cd8_neg_CTLs_interacts<- c(B_cell_cd3_pos_cd8_neg_CTLs_interacts, inverse_interact)
    }
}

cd3_pos_cd8_neg_CTLs_B_cell_interacts
sum(colnames(ph_wt_6mos_pvals) %in% cd3_pos_cd8_neg_CTLs_B_cell_interacts) == length(cd3_pos_cd8_neg_CTLs_B_cell_interacts)

B_cell_cd3_pos_cd8_neg_CTLs_interacts
sum(colnames(ph_wt_6mos_pvals) %in% B_cell_cd3_pos_cd8_neg_CTLs_interacts) == length(B_cell_cd3_pos_cd8_neg_CTLs_interacts)

[1] "cd3_pos_cd8_neg_CTLs|B_cell_Atypical"                 
[2] "cd3_pos_cd8_neg_CTLs|B_cell_Age-associated/autoimmune"
[3] "cd3_pos_cd8_neg_CTLs|B_cell_Cycling"                  
[4] "cd3_pos_cd8_neg_CTLs|B_cell_Plasmablasts"

[1] TRUE

[1] "B_cell_Atypical|cd3_pos_cd8_neg_CTLs"                 
[2] "B_cell_Age-associated/autoimmune|cd3_pos_cd8_neg_CTLs"
[3] "B_cell_Cycling|cd3_pos_cd8_neg_CTLs"                  
[4] "B_cell_Plasmablasts|cd3_pos_cd8_neg_CTLs"

[1] TRUE

In [43]:
cd3_pos_cd8_neg_B_cell_mtx_bcl6 = build_cpdb_mtx_noSick(cols = cd3_pos_cd8_neg_CTLs_B_cell_interacts,
                                            scaled = FALSE,
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1,  
                                            group_name = "_bcl6",
                                            ph_6mos_pvals = ph_bcl6_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_18mos_means)

B_cell_cd3_pos_cd8_neg_mtx_bcl6 = build_cpdb_mtx_noSick(cols = B_cell_cd3_pos_cd8_neg_CTLs_interacts, 
                                            scaled = FALSE,
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1,  
                                            group_name = "_bcl6",
                                            ph_6mos_pvals = ph_bcl6_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_18mos_means)

cd3_pos_cd8_neg_B_cell_mtx_bcl6_double = build_cpdb_mtx_noSick(cols = cd3_pos_cd8_neg_CTLs_B_cell_interacts,
                                            scaled = FALSE,
                                            group_name = "_bcl6_double",
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_bcl6_double_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_double_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_double_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_double_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_double_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_double_18mos_means)

B_cell_cd3_pos_cd8_neg_mtx_bcl6_double = build_cpdb_mtx_noSick(cols = B_cell_cd3_pos_cd8_neg_CTLs_interacts, 
                                            scaled = FALSE,
                                            group_name = "_bcl6_double",
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_bcl6_double_6mos_pvals,
                                            ph_6mos_means = ph_bcl6_double_6mos_means,
                                            ph_14mos_pvals = ph_bcl6_double_14mos_pvals,
                                            ph_14mos_means = ph_bcl6_double_14mos_means,
                                            ph_18mos_pvals = ph_bcl6_double_18mos_pvals,
                                            ph_18mos_means = ph_bcl6_double_18mos_means)

cd3_pos_cd8_neg_B_cell_mtx_wt = build_cpdb_mtx_wt(cols = cd3_pos_cd8_neg_CTLs_B_cell_interacts,
                                            scaled = FALSE,
                                            group_name = "_wt",
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_wt_6mos_pvals,
                                            ph_6mos_means = ph_wt_6mos_means,
                                            ph_14mos_pvals = ph_wt_14mos_pvals,
                                            ph_14mos_means = ph_wt_14mos_means,
                                            ph_18mos_pvals = ph_wt_18mos_pvals,
                                            ph_18mos_means = ph_wt_18mos_means)

B_cell_cd3_pos_cd8_neg_mtx_wt = build_cpdb_mtx_wt(cols = B_cell_cd3_pos_cd8_neg_CTLs_interacts, 
                                            scaled = FALSE,
                                            group_name = "_wt",
                                            sig.cellpairs = 3,
                                            sig.timepoint = 1, 
                                            ph_6mos_pvals = ph_wt_6mos_pvals,
                                            ph_6mos_means = ph_wt_6mos_means,
                                            ph_14mos_pvals = ph_wt_14mos_pvals,
                                            ph_14mos_means = ph_wt_14mos_means,
                                            ph_18mos_pvals = ph_wt_18mos_pvals,
                                            ph_18mos_means = ph_wt_18mos_means)

total number of intersections considered: 28

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 22

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 23

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 22

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 24

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data

total number of intersections considered: 20

6 mos...

14 mos...

18 mos...

Generating count matrix...

Generating pvalue matrix...

output unscaled data



In [44]:
cd3_pos_cd8_neg_CTLs_B_cell_mtx<- merge_mtx(cd3_pos_cd8_neg_B_cell_mtx_wt, cd3_pos_cd8_neg_B_cell_mtx_bcl6, cd3_pos_cd8_neg_B_cell_mtx_bcl6_double)
B_cell_cd3_pos_cd8_neg_CTLs_mtx<- merge_mtx(B_cell_cd3_pos_cd8_neg_mtx_wt, B_cell_cd3_pos_cd8_neg_mtx_bcl6, B_cell_cd3_pos_cd8_neg_mtx_bcl6_double)

wt...

bcl6...

bcl6 double...



[1] NA NA


TRUE

scaling the count matrix

wt...

bcl6...

bcl6 double...



[1] NA NA


TRUE

scaling the count matrix



In [46]:
hp3 = plt_cpdb_dotplot(exp_mtx = cd3_pos_cd8_neg_CTLs_B_cell_mtx$exp_mtx, 
                       pval_mtx = cd3_pos_cd8_neg_CTLs_B_cell_mtx$pval_mtx)
hp4 = plt_cpdb_dotplot(exp_mtx = B_cell_cd3_pos_cd8_neg_CTLs_mtx$exp_mtx, 
                       pval_mtx = B_cell_cd3_pos_cd8_neg_CTLs_mtx$pval_mtx)
pdf("./results/figures/9_Dotplot_cd3_pos_cd8_neg_CTLs_B_cell_wt_bcl6_and_bcl6_double_cell3_time1_genotype1.pdf", width = 12, height = 10)
draw(hp3, annotation_legend_list = lgd, ht_gap = unit(1, "cm"))
dev.off()

pdf("./results/figures/9_Dotplot_B_cell_cd3_pos_cd8_neg_CTLs_wt_bcl6_and_bcl6_double_cell3_time1_genotype1.pdf", width = 12, height = 10)
draw(hp4, annotation_legend_list = lgd, ht_gap = unit(1, "cm"))
dev.off()

quantile: -1.17342769428179-0.1667917315923481.88143360605578



genotype
       bcl6 bcl6_double          wt 
         12          12          12 
age
06mos 14mos 18mos 
   12    12    12 


quantile: -1.14696172974392-0.2813343844440211.90250182175104



genotype
       bcl6 bcl6_double          wt 
         12          12          12 
age
06mos 14mos 18mos 
   12    12    12 


png 
  2

png 
  2

In [ ]:
# i. CTLA4-CD80
# ii. CTLA4-CD86
# iii. PDCD1-PDCD1LG2
# iv. PDCD1-CD274
# v. Lgals9-Havcr2

In [47]:
cd3_pos_cd8_neg_B_cell_interactions<- c("CTLA4_CD80", "CTLA4_CD86", "PDCD1_PDCD1LG2", "PDCD1_CD274")

cd3_pos_cd8_neg_B_cell_exp_mtx_sub<- cd3_pos_cd8_neg_CTLs_B_cell_mtx$exp_mtx[cd3_pos_cd8_neg_B_cell_interactions, ]
cd3_pos_cd8_neg_B_cell_pval_mtx_sub<- cd3_pos_cd8_neg_CTLs_B_cell_mtx$pval_mtx[cd3_pos_cd8_neg_B_cell_interactions, ]

cd3_pos_cd8_neg_B_cell_mtx_sub = list("exp_mtx" = cd3_pos_cd8_neg_B_cell_exp_mtx_sub, "pval_mtx" = cd3_pos_cd8_neg_B_cell_pval_mtx_sub)

hp_sub = plt_cpdb_dotplot(exp_mtx = cd3_pos_cd8_neg_B_cell_mtx_sub$exp_mtx, 
                           pval_mtx = cd3_pos_cd8_neg_B_cell_mtx_sub$pval_mtx)

pdf("./results/figures/9_Dotplot_cd3_pos_cd8_neg_B_cell_wt_bcl6_and_bcl6_double_cell3_time1_genotype1_subset.pdf", width = 12, height = 2)
draw(hp_sub, annotation_legend_list = lgd, ht_gap = unit(1, "cm"))
dev.off()

quantile: -1.19239009499975-0.04021837888061971.95888813308765



genotype
       bcl6 bcl6_double          wt 
         12          12          12 
age
06mos 14mos 18mos 
   12    12    12 


png 
  2